### Import library

In [1]:
import os
os.chdir('../..')
os.getcwd()

'C:\\Users\\csia7\\OneDrive\\문서\\GitHub\\WQBrain_2024_API'

In [2]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import pygwalker as pyg
import glob
import re

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [3]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_TfhaFpZ4WCY8LdHeReaAAD5pAzBm

 


In [4]:
ace.check_session_timeout(s)

14398.93859

## Alpha Mixing Templates

## Read all passed alphas

In [6]:
os.chdir('Results/signal')
file_direc = glob.glob('*')
file_direc

['A_signalform.csv',
 'B_signalform.csv',
 'C_signalform.csv',
 'D_signalform.csv',
 'pre_signalform.csv',
 'Sep03GLB_signalform.csv']

In [7]:
expressions = {}
for file in file_direc:
    expressions[file.split('_')[0]] = pd.read_csv(file)['expression']

In [8]:
expressions.keys()

dict_keys(['A', 'B', 'C', 'D', 'pre', 'Sep03GLB'])

## Template B & Sep03GLB mixing
use top 32 results

In [9]:
A = expressions['A'][:32]
B = expressions['B']
Sep03GLB = expressions['Sep03GLB']

In [12]:
alpha_list_BSep03GLB = []

for i in range(2):
    for j in range(2):
        alpha_list_BSep03GLB += [ace.generate_alpha(f'{B[i]};{Sep03GLB[j]};add(scale_down(signal_B_{i}), scale_down(signal_Sep03GLB_{j}), filter = false)', region= "GLB", universe = "MINVOL1M", neutralization = x, truncation = 0.01, decay = 3) for x in ['COUNTRY', 'SECTOR']]

alpha_list_BSep03GLB

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 3,
   'neutralization': 'COUNTRY',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'entry_B_0 = ts_rank( ts_std_dev(oth460_3l_vlc, 21), 252 ); maintenance_B_0 = -ts_regression(rsk70_mfm2_gemtrd_srtindcnt, ts_delay(rsk70_mfm2_gemtrd_anlystsn, 1), 252); quit_B_0 = entry_B_0 < 0.5;signal_B_0 =  trade_when(entry_B_0>0.55, maintenance_B_0, quit_B_0);group_Sep03GLB_0 = (country+1)*group_max(pv13_52_minvol_1m_all_delay_1_sector, market) + pv13_52_minvol_1m_all_delay_1_sector;signal_Sep03GLB_0 = group_neutralize(ts_scale(group_backfill(fnd23_intfvalld1_ecns, industry, 252, std=1), 252), densify(group_Sep03GLB_0));add(scale_down(signal_B_0), scale_down(signal_Sep03GLB_0), filter = false)'},
 {'type': 'REGULAR'

In [13]:
result = ace.simulate_alpha_list_multi(s, alpha_list_BSep03GLB)

Warning, list of alphas too short, single concurrent simulations will be used instead of multisimulations


100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [19:45<00:00, 148.14s/it]


In [14]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,start_date,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,6993493,20000000,1778,1718,0.2925,0.0675,0.0171,0.000462,1.68,3.50,2012-01-22,dxGpLJE,"entry_B_0 = ts_rank( ts_std_dev(oth460_3l_vlc,...",PASS,PASS,PASS,PASS,PASS,PASS,PASS
1,5791844,20000000,1689,1617,0.2378,0.0559,0.0270,0.000470,1.29,2.66,2012-01-22,dxGpwPg,entry_B_1 = ts_rank( ts_std_dev(oth460_1l_1dls...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
2,7867241,20000000,1773,1722,0.2925,0.0760,0.0305,0.000519,1.21,2.37,2012-01-22,27YMdgJ,"entry_B_0 = ts_rank( ts_std_dev(oth460_3l_vlc,...",PASS,PASS,PASS,PASS,PASS,PASS,PASS
3,7586483,20000000,1683,1623,0.2397,0.0733,0.0347,0.000611,1.18,2.14,2012-01-22,Wp5MmxZ,entry_B_1 = ts_rank( ts_std_dev(oth460_1l_1dls...,PASS,PASS,PASS,PASS,PASS,PASS,PASS


In [15]:
result_st1.to_csv('B&Sep03GLB_mixing_add.csv')

In [16]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df.head()

,alpha_id,endDate,limit,message,name,result,startDate,themes,value,year
0,27YMdgJ,NaN,1.58,NaN,LOW_SHARPE,PASS,NaN,NaN,2.3700,NaN
1,27YMdgJ,NaN,1.00,NaN,LOW_FITNESS,PASS,NaN,NaN,1.2100,NaN
2,27YMdgJ,NaN,0.01,NaN,LOW_TURNOVER,PASS,NaN,NaN,0.2925,NaN
3,27YMdgJ,NaN,0.70,NaN,HIGH_TURNOVER,PASS,NaN,NaN,0.2925,NaN
4,27YMdgJ,NaN,NaN,NaN,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,NaN


In [17]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:[]
Passed alphas:['27YMdgJ', 'dxGpwPg', 'Wp5MmxZ', 'dxGpLJE']


In [19]:
#calling submit_alpha on all alphas that have passed the submission tests
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['Sep05_mixing'])

In [37]:
submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas[]}
submit_result

{'5wmQEZo': True, 'g1lQeJK': True}

### Library Fuctions.

following are some other functions that you can use for your own analysis

**get_alpha_pnl(s, alpha_id)** - to get the pnl for an alpha

**get_alpha_yearly_stats(s, alpha_id)** - to get yearly statistics for an alpha

**get_self_corr(s, alpha_id)** - to get self correlation results for an alpha

**get_prod_corr(s, alpha_id)** - to get prod correlation results for an alpha

**get_check_submission(s, alpha_id)** - to get check submission result for an alpha

**check_self_corr_test(s, alpha_id)** - to check if alpha passes self correlation test (self_corr<0.7)

**check_prod_corr_test(s, alpha_id)** - to check if alpha passes prod correlation test (prod_corr<0.7)

**perfomance_comparison(s, alpha_id)** - to get the result of performance comparison for an alpha merged performance

In [37]:
ace.get_alpha_pnl(s, result[0]['alpha_id'])

,Pnl,alpha_id
Date,,
2012-01-23,0.0,dx9ALqJ
2012-01-24,52190.0,dx9ALqJ
2012-01-25,32038.0,dx9ALqJ
2012-01-26,37403.0,dx9ALqJ
2012-01-27,53631.0,dx9ALqJ
...,...,...
2022-01-17,12935472.0,dx9ALqJ
2022-01-18,12935661.0,dx9ALqJ
2022-01-19,12967474.0,dx9ALqJ
